_This project will be used for the Capstone_

In [2]:
import pandas as pd
import numpy as np


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<font size="5">Introduction/Business Problem Section</font>

Problem:Compare the neighborhoods within New York City and determine key insights that will improve the quality of life.

Audience: My audience would be anyone who is living, wants to live in, or previously lived in New York City.
Why they would care: They might be interested in analyzing what to better improve in order to improve the quality of life. 

<font size="5">Data Section</font>

I will be using Foursquare location data. I will also use furmancenter.org for data about New York City Neighborhoods.

In [ ]:
The Foursquare Data about New York City is the one that we used in the Project already. The reason I am using furmancenter.org is that the 